In [1]:
import os

import torch
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchvision import transforms
from torchvision.transforms import InterpolationMode

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': './Dataset/Albedo/train',
    'test_dir': './Dataset/Albedo/test',
    
    # Cấu hình train
    'backbone': 'iresnet18',
    'epochs': 2000,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('./Dataset/Albedo/train')),
    'embedding_size': 512,
    
    'learning_rate': 0.2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

In [3]:
# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

In [4]:
train_transform = transforms.Compose([
    transforms.RandomCrop(224),
])

test_transform = transforms.Compose([
    transforms.Resize((224,224), interpolation=InterpolationMode.BILINEAR)
])

In [5]:
train_dataloader, val_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

In [6]:
model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=40, T_mult=1, eta_min=1e-6)
checkpoint_path = os.path.abspath('checkpoint/magface/albedo/' + CONFIGURATION['backbone'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/albedo/' + CONFIGURATION['backbone'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['val_euclidean_accuracy', 'val_cosine_accuracy', 'val_auc_euclidean', 'val_auc_cosine'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=40
)            

In [7]:
fit(CONFIGURATION, 0, model, device, train_dataloader, val_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 1:
	train: loss 3.688 | loss id   3.66 | top_1_acc 0.0030 | top_5_acc 0.0239 | acc_eu: 0.452 | acc_cos: 0.517 | auc_eu: 0.620 | auc_cos: 0.631
	val: acc_eu: 0.624 | acc_cos: 0.568 | auc_eu: 0.614 | auc_cos: 0.636
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/albedo/iresnet18/models/checkpoint.pth
Epoch 2:
	train: loss 1.987 | loss id   1.96 | top_1_acc 0.0052 | top_5_acc 0.0299 | acc_eu: 0.565 | acc_cos: 0.556 | auc_eu: 0.652 | auc_cos: 0.662
	val: acc_eu: 0.588 | acc_cos: 0.623 | auc_eu: 0.667 | auc_cos: 0.681
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/albedo/iresnet18/models/checkpoint.pth
Epoch 3:
	train: loss 0.651 | loss id   0.63 | top_1_acc 0.0090 | top_5_acc 0.0336 | acc_eu: 0.569 | acc_cos: 0.594 | auc_eu: 0.648 | auc_cos: 0.683
	val: acc_eu: 0.621 | acc_cos: 0.518 | auc_eu: 0.625 | auc_cos: 0.654
	Saving mod